In [1]:
#Importing all these tools.  I usually need one or the other.
#For now lets just grab them all.  Some might not be used.
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import explore
import warnings
warnings.filterwarnings("ignore")

### Given the following confusion matrix, evaluate (by hand) the model's performance.

In [2]:
data = {'actual_dog':['46','7'],'actual_cat':['13','34']}
df = pd.DataFrame(data, columns = ['actual_dog','actual_cat'], index=['pred_dog','pred_cat'])
df

,actual_dog,actual_cat
pred_dog,46,13
pred_cat,7,34


#### In the context of this problem, what is a false positive?
    False positive would be Not a Dog but predicted a Dog
#### In the context of this problem, what is a false negative?
    False negative is that IS a Dog but predicted a Cat
#### How would you describe this model?
    Crosstab of predicting dogs and cats

### You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

Use the predictions dataset and pandas to help answer the following questions:

#### An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

#### Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?